# Will it be delayed?

Everyone who has flown has experienced a delayed or cancelled flight. Both airlines and airports would like to improve their on-time performance and predict when a flight will be delayed or cancelled several days in advance. You are being hired to build a model that can predict if a flight will be delayed. To learn more, you must schedule a meeting with your client (me). To schedule an appointment with your client, send an event request through Google Calendar for a 15 minute meeting. Both you and your project partner must attend the meeting. Come prepared with questions to ask your client. Remember that your client is not a data scientist and you will need to explain things in a way that is easy to understand. Make sure that your communications are efficient, thought out, and not redundant as your client might get frustrated and "fire" you (this only applies to getting information from your client, this does not necessary apply to asking for help with the actual project itself - you should continuously ask questions for getting help).

For this project you must go through most all steps in the checklist. You must write responses for all items as done in the homeworks, however sometimes the item will simply be "does not apply". Keep your progress and thoughts organized in this document and use formatting as appropriate (using markdown to add headers and sub-headers for each major part). Some changes to the checklist:

* Do not do the final part (launching the product).
* Your presentation will be done as information written in this document in a dedicated section (no slides or anything like that). It should include high-level summary of your results (including what you learned about the data, the "accuracy" of your model, what features were important, etc). It should be written for your client, not your professor or teammates. It should include the best summary plots/graphics/data points.
* The models and hyperparameters you should consider during short-listing and fine-tuning will be released at a later time (dependent on how far we get over the next two weeks).
* Data retrieval must be automatic as part of the code (so it can easily be re-run and grab the latest data). Do not commit any data to the repository.
* Your submission must include a pickled final model along with this notebook.

In [1]:
#Imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 

Get the Data
============

1. **List the data you need and how much you need:**
    - National flight data for 2023 and 2024
    - Weather data covering all of the same dates, preferably daily.
2. **Find and document where you can get that data:**
    - All of the weather data is available on the NOAA website. You must go through and make an order for each individual airport. The link is here: https://www.ncei.noaa.gov/cdo-web/ 
    - Flight data is from: https://www.transtats.bts.gov/tables.asp?QO_VQ=EFD&QO_anzr=Nv4yv0r 
3. **Get access authorizations**:
   - You must agree to the terms of use and make an order (which is free for digital use).
4. **Create a workspace**: This notebook.
5. **Get the data**: 
    - Download all of the CSV files from the websites mentioned above
6. **Convert the data to a format you can easily manipulate**:
   - The data is all in one parquet file.

In [2]:
data = pd.read_parquet('combined.parquet')
columns_to_keep = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'OriginAirportID', 'Origin', 'OriginCityName', 'OriginStateName' ,'DestAirportID', 'Dest', 'DestCityName', 'DestStateName', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'ArrTime', 'ArrDelayMinutes', 'Cancelled', 'CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'AirTime', 'Flights', 'Distance']
data = data[columns_to_keep]

In [3]:
weather_df = pd.read_csv('3964079.csv')

In [4]:
# Extract the unique names from the 'NAME' column
unique_names = weather_df['NAME'].unique()

# Print the distinct names
print("Distinct names in the weather data:")
for name in unique_names:
    print(name)

Distinct names in the weather data:
MIAMI INTERNATIONAL AIRPORT, FL US
CHICAGO OHARE INTERNATIONAL AIRPORT, IL US
LOS ANGELES INTERNATIONAL AIRPORT, CA US
DENVER INTERNATIONAL AIRPORT, CO US
ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US
SEATTLE TACOMA AIRPORT, WA US
DAL FTW WSCMO AIRPORT, TX US


In [5]:
print("Weather dataset columns:", weather_df.columns)
print(weather_df.head())

Weather dataset columns: Index(['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD',
       'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02',
       'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10'],
      dtype='object')
       STATION                                NAME        DATE   AWND  PGTM  \
0  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-01   4.47   NaN   
1  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-02   7.16   NaN   
2  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-03  10.51   NaN   
3  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-04   8.50   NaN   
4  USW00012839  MIAMI INTERNATIONAL AIRPORT, FL US  2023-01-05   5.37   NaN   

   PRCP  SNOW  SNWD  TAVG  TMAX  ...  WT01  WT02  WT03  WT04  WT05  WT06  \
0   0.0   NaN   NaN    76  86.0  ...   1.0   NaN   NaN   NaN   NaN   NaN   
1   0.0   NaN   NaN    76  84.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   
2   0.0   NaN   NaN    77

In [6]:
# --------------------------
# Process Weather Data
# --------------------------

# Convert 'DATE' to datetime and extract date components
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_df['Year'] = weather_df['DATE'].dt.year
weather_df['Month'] = weather_df['DATE'].dt.month
weather_df['DayofMonth'] = weather_df['DATE'].dt.day

# Rename 'STATION' to 'WeatherStation' for clarity
weather_df.rename(columns={'STATION': 'WeatherStation'}, inplace=True)

# Optional: print date range to verify
print("Weather date range:", weather_df['DATE'].min(), weather_df['DATE'].max())

# --------------------------
# Process Flight Data
# --------------------------
# Assume flight data is loaded into a DataFrame called 'data'
data['FlightDate'] = pd.to_datetime(data['FlightDate'])

# (If not already available, ensure these columns exist: Year, Month, DayofMonth)
# If they aren't already present in the flight data, you can extract them from FlightDate:
# data['Year'] = data['FlightDate'].dt.year
# data['Month'] = data['FlightDate'].dt.month
# data['DayofMonth'] = data['FlightDate'].dt.day

# --------------------------
# Create Mapping from Airport Code to Weather Station ID
# --------------------------
airport_to_station = {
    'ATL': 'USW00013874',
    'ORD': 'USW00094846',
    'SEA': 'USW00024233',
    'MIA': 'USW00012839',
    'DFW': 'USW00003927',
    'LAX': 'USW00023174',
    'DEN': 'USW00003017',
    # Add additional mappings as needed...
}

# Create a new column in flight data that contains the mapped WeatherStation ID
data['WeatherStation'] = data['Origin'].map(airport_to_station)

# --------------------------
# Merge the Datasets
# --------------------------
combined_df = pd.merge(
    data,
    weather_df,
    on=['Year', 'Month', 'DayofMonth', 'WeatherStation'],
    how='left'
)

# Preview the merged dataset
print("Combined dataset preview:")
print(combined_df.head())

Weather date range: 2023-01-01 00:00:00 2024-12-31 00:00:00
Combined dataset preview:
   Year  Month  DayofMonth  DayOfWeek FlightDate  OriginAirportID Origin  \
0  2023     12          30          6 2023-12-30            12339    IND   
1  2023     12          30          6 2023-12-30            12953    LGA   
2  2023     12           1          5 2023-12-01            12953    LGA   
3  2023     12           3          7 2023-12-03            12953    LGA   
4  2023     12           4          1 2023-12-04            12953    LGA   

     OriginCityName OriginStateName  DestAirportID  ... WT01 WT02 WT03  WT04  \
0  Indianapolis, IN         Indiana          12953  ...  NaN  NaN  NaN   NaN   
1      New York, NY        New York          12339  ...  NaN  NaN  NaN   NaN   
2      New York, NY        New York          15016  ...  NaN  NaN  NaN   NaN   
3      New York, NY        New York          15016  ...  NaN  NaN  NaN   NaN   
4      New York, NY        New York          15016  ...  

7. **Ensure sensitive information is deleted or protected**: This is public data
8. **Check the size and type of data (time series, geographical, …)**:

<mark>TODO</mark>: report your information below. At this point, since you don't want to look at the data too closely, this is a quick evaluation about the number of features and their data types (note: remember that just because all values for a feature are a number doesn't mean that feature is numerical), the number of samples (including possible missing data), and any special considerations about the features such as:

   1. Is it a time series: 
      - Yes

   2. Are any of the features unusable for the business problem? Or are some not available for the business problem when the model will be used?: 
      - Yes, so far all of the features are usable except for diverted flights, because the client doesn't want those accounted

   3. Which feature(s) will be used as the target/label for the business problem? (including which are required to derive the correct label)
      

   4. Should any of the features be stratified during the train/test split to avoid sampling biases?
   

Do not look at the data too closely at this point since you have not yet split off the testing set. Basically, enough looking at it to understand *how* to split the test set off. It is likely you will have to review the website where the data came from to be able to understand some of the features.

Frame the Problem and Look at the Big Picture
=============================================


1. **Define the objective in business terms:** 
2. **How will your solution be used?**
3. **What are the current solutions/workarounds (if any)?** 
4. **How should you frame this problem?** 
5. **How should performance be measured? Is the performance measure aligned with the business objective?** *Don't worry about this for now*
6. **What would be the minimum performance needed to reach the business objective?** *Don't worry about this for now*
7. **What are comparable problems? Can you reuse (personal or readily available) experience or tools?** 
8. **Is human expertise available?** Not immediately.
9. **How would you solve the problem manually?** 
10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.** 

In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14825707 entries, 0 to 14825706
Data columns (total 53 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Year               int64         
 1   Month              int64         
 2   DayofMonth         int64         
 3   DayOfWeek          int64         
 4   FlightDate         datetime64[ns]
 5   OriginAirportID    int64         
 6   Origin             object        
 7   OriginCityName     object        
 8   OriginStateName    object        
 9   DestAirportID      int64         
 10  Dest               object        
 11  DestCityName       object        
 12  DestStateName      object        
 13  DepTime            float64       
 14  DepDelay           float64       
 15  DepDelayMinutes    float64       
 16  ArrTime            float64       
 17  ArrDelayMinutes    float64       
 18  Cancelled          float64       
 19  CancellationCode   object        
 20  CarrierDelay       flo

In [8]:
combined_df.describe()

,Year,Month,DayofMonth,DayOfWeek,FlightDate,OriginAirportID,DestAirportID,DepTime,DepDelay,DepDelayMinutes,...,WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10
count,1.482571e+07,1.482571e+07,1.482571e+07,1.482571e+07,14825707,1.482571e+07,1.482571e+07,1.463667e+07,1.463630e+07,1.463630e+07,...,1103623.0,140480.0,473317.0,13857.0,24109.0,13408.0,5178.0,483125.0,5416.0,1047.0
mean,2.023509e+03,6.586377e+00,1.577153e+01,3.983661e+00,2024-01-06 02:34:45.840818944,1.265432e+04,1.265433e+04,1.332564e+03,1.235834e+01,1.565029e+01,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,2.023000e+03,1.000000e+00,1.000000e+00,1.000000e+00,2023-01-01 00:00:00,1.013500e+04,1.013500e+04,1.000000e+00,-9.900000e+01,0.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,2.023000e+03,4.000000e+00,8.000000e+00,2.000000e+00,2023-07-08 00:00:00,1.129200e+04,1.129200e+04,9.120000e+02,-6.000000e+00,0.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,2.024000e+03,7.000000e+00,1.600000e+01,4.000000e+00,2024-01-07 00:00:00,1.288900e+04,1.288900e+04,1.325000e+03,-2.000000e+00,0.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,2.024000e+03,1.000000e+01,2.300000e+01,6.000000e+00,2024-07-08 00:00:00,1.402700e+04,1.402700e+04,1.746000e+03,9.000000e+00,9.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,2.024000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2024-12-31 00:00:00,1.686900e+04,1.686900e+04,2.400000e+03,5.764000e+03,5.764000e+03,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,4.999182e-01,3.403419e+00,8.781058e+00,2.007278e+00,NaN,1.526151e+03,1.526147e+03,5.077571e+02,5.612929e+01,5.507429e+01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
combined_df.head()

,Year,Month,DayofMonth,DayOfWeek,FlightDate,OriginAirportID,Origin,OriginCityName,OriginStateName,DestAirportID,...,WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10
0,2023,12,30,6,2023-12-30,12339,IND,"Indianapolis, IN",Indiana,12953,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,12,30,6,2023-12-30,12953,LGA,"New York, NY",New York,12339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,12,1,5,2023-12-01,12953,LGA,"New York, NY",New York,15016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,12,3,7,2023-12-03,12953,LGA,"New York, NY",New York,15016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,12,4,1,2023-12-04,12953,LGA,"New York, NY",New York,15016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
